In [9]:
# import initial dependencies
import pandas as pd
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from pprint import pprint
from api_key import omdb_key
import scipy.stats as stats

In [2]:
# create a path to academy awards csv and read it into a pandas dataframe
awards_csv = "academy_awards_data_2.csv"
awards_df = pd.read_csv(awards_csv, usecols = ['Nominee', 'Year', 'Category', 'Won?'], encoding = 'latin-1')

# create another dataframe that only includes nominees in the Best Picture category
award_data = awards_df.loc[awards_df["Category"] == "Best Picture", :]

# create list of best picture nominees
best_picture_noms = award_data["Nominee"]

# print(best_picture_noms)
# print(best_picture_noms[77])
# award_data.head()

In [ ]:
# NO NEED TO RUN THIS CELL AGAIN BECAUSE DATA IS ALREADY READ INTO CSV BELOW, 
# BUT TOTALLY FEEL FREE TO TEST IF YOU'VE INCLUDED ADDITIONAL/DIFFERENT AWARD CATEGORIES

# print the corresponding number for each movie  
movie_number = 1

# empty lists for holding movie data
box_office = []
genre = []
meta_score = []
imdb_rating = []
title = []
poster_url = []
rated = []
release_date = []
studio = []

best_picture_noms = award_data["Nominee"]
base_url = "http://www.omdbapi.com/?"

    
# print statement as each movie is processed
print(f"Beginning Data Retrieval")
print(f"==============================")

# loop through the movies in the best picture noms dataframe 
for movie in best_picture_noms:
    
    params = {
    "apikey" : omdb_key,
    "t" : movie
    } 
    
    # try statement for each potential movie
    try: 
        omdb_data_raw = requests.get(base_url, params=params)
        omdb_data = omdb_data_raw.json()
        box_office.append(omdb_data["BoxOffice"])
        genre.append(omdb_data["Genre"])
        meta_score.append(omdb_data["Metascore"])
        imdb_rating.append(omdb_data["imdbRating"])
        title.append(omdb_data["Title"])
        poster_url.append(omdb_data["Poster"])
        rated.append(omdb_data["Rated"]) 
        release_date.append(omdb_data["Released"]) 
        studio.append(omdb_data["Production"])
        print_title = omdb_data["Title"]
        
        print(f"Processing Record {movie_number} | {print_title}")
        print(omdb_data_raw.url) 
        
        # increase movie number by one each loop
        movie_number = movie_number + 1
        
#         to avoid 60 rpm api limit i'm waiting just over 1 second per loop
#         https://www.pythoncentral.io/pythons-time-sleep-pause-wait-sleep-stop-your-code/)
        time.sleep(1.01)
        
    # skip if no movie is found or if data is missing
    except:
        print("Data missing or movie not found. Skipping...")
    continue
    
print(f"==============================")
print(f"Data Retrieval Complete")
print(f"==============================")

In [ ]:
# NO NEED TO RUN THIS CELL AGAIN BECAUSE DATA IS ALREADY READ INTO CSV BELOW, 
# BUT TOTALLY FEEL FREE TO TEST IF YOU'VE INCLUDED ADDITIONAL/DIFFERENT AWARD CATEGORIES

# converting filtered api data into dataframe
filtered_omdb_data_df= pd.DataFrame ({
    "Title": title,
    "Genre": genre,
    "Meta_Score": meta_score,
    "imdb_Rating": imdb_rating,
    "Box_Office" : box_office,
    "Rated" : rated,
    "Studio" : studio,
    "Release_Date" : release_date,
    "Poster_URL" : poster_url
})

# coverting dataframe into csv-- this step isn't necessary, but did it so i'm not constantly dealing with the api directly
filtered_omdb_data_df.to_csv('filtered_omdb_data.csv', index=False)
# filtered_omdb_data_df.head()

In [15]:
# created a path to the filteredd api csv and read it into a pandas dataframe
filtered_omdb_csv = "filtered_omdb_data.csv"
filtered_omdb_csv_df = pd.read_csv(filtered_omdb_csv)
# filtered_omdb_csv_df.count()
# award_data.count()

# merged the filtered api data csv and awards data csv into a single dataset
merged_movie_data_df = pd.merge(filtered_omdb_csv_df, award_data, left_on="Title", right_on="Nominee")
# merged_movie_data_df.to_csv('merged_movie_data.csv', index=False)
# merged_movie_data_df.head()

# import Eric's csv that adds a seasons column to "merged_movie_data_df"
seasons_omdb_csv = "Seasons_Movie_Data_2.csv"
seasons_omdb_csv_df = pd.read_csv(seasons_omdb_csv)
# seasons_omdb_csv_df.head()

In [16]:
# clean up Rated category.
seasons_omdb_csv_df['Rated'] = seasons_omdb_csv_df['Rated'].replace(
    {'NOT RATED': 'Not Rated', 'PASSED': 'Passed', 'UNRATED': 'Not Rated', 'Unrated': 'Not Rated', 'APPROVED': 'Approved'})

In [17]:
# create dataframe for winning and losing nominees
winning_noms = seasons_omdb_csv_df[seasons_omdb_csv_df["Won?"] == "YES"]
# winning_noms

losing_noms = seasons_omdb_csv_df[seasons_omdb_csv_df["Won?"] == "NO"]
# losing_noms

In [18]:
winning_noms_imdb = winning_noms['imdb_Rating']
losing_noms_imdb = losing_noms['imdb_Rating']

stats.ttest_ind(winning_noms_imdb, losing_noms_imdb, equal_var=False)

Ttest_indResult(statistic=3.5733867271080175, pvalue=0.0005327364755244779)

In [ ]:
#count of rated (R, PG, G, etc.) comparison nominees Won vs. Lost 

# set width of bar
barWidth = 0.25
 
# set height of bar
height_winning_rated = [22, 19, 12, 9, 7, 2, 6]
height_losing_rated = [89, 86, 67, 38, 39, 42, 20]

 
# Set position of bar on X axis
r1 = np.arange(len(height_winning_rated))
r2 = [x + barWidth for x in r1]
 
# Make the plot
plt.bar(r1, height_winning_rated, color='navy', width=barWidth, edgecolor='white', label='Winning Noms')
plt.bar(r2, height_losing_rated, color='orange', width=barWidth, edgecolor='white', label='Losing Noms')
 
# Add xticks on the middle of the group bars
plt.title("'Best Picture' Nominee Movie Rating (1927-2010)", fontweight='bold')
plt.xlabel('Movie Rating', fontweight='bold')
plt.ylabel("Count of Movie Rating", fontweight='bold')

plt.xticks([r + barWidth for r in range(len(height_winning_rated))], ['R', 'Not Rated', 'PG', 'PG-13', 'Passed', 'Approved', 'G'])

#create legend, show graphic, and push to .png
plt.legend()
plt.savefig("count_of_rated_grouped.png")
plt.show()

In [ ]:
winter_noms = seasons_omdb_csv_df[seasons_omdb_csv_df["Seasons"] == "Winter"]
spring_noms = seasons_omdb_csv_df[seasons_omdb_csv_df["Seasons"] == "Spring"]
summer_noms = seasons_omdb_csv_df[seasons_omdb_csv_df["Seasons"] == "Summer"]
fall_noms = seasons_omdb_csv_df[seasons_omdb_csv_df["Seasons"] == "Fall"]

winter_rated = winter_noms['Rated'].value_counts()
# print(winter_rated)

spring_rated = spring_noms['Rated'].value_counts()
# print(spring_rated)

summer_rated = summer_noms['Rated'].value_counts()
# print(summer_rated)

fall_rated = fall_noms['Rated'].value_counts()
# print(fall_rated)

# rated_as_index_ = merged_movie_data_df.set_index('Rated').groupby(['Rated'])
# rated_as_index.head()

In [ ]:
# count of rated (R, PG, G, etc.) by season

# set width of bar
barWidth = 0.25

# set height of bar
height_winter_rated = [60, 36, 30, 25, 11, 7, 4]
height_spring_rated = [12, 12, 22, 5, 10, 13, 5]
height_summer_rated = [14, 16, 19, 7, 13, 10, 10]
height_fall_rated = [25, 15, 30, 10, 10, 16, 7]

 
# Set position of bar on X axis
r1 = np.arange(len(height_winter_rated))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]
 
# Make the plot
plt.bar(r1, height_winter_rated, color='navy', width=barWidth, edgecolor='white', label='Winter')
plt.bar(r2, height_spring_rated, color='orange', width=barWidth, edgecolor='white', label='Spring')
plt.bar(r3, height_summer_rated, color='red', width=barWidth, edgecolor='white', label='Summer')
plt.bar(r4, height_fall_rated, color='black', width=barWidth, edgecolor='white', label='Fall')

 
# Add xticks on the middle of the group bars
plt.title("'Best Picture' Nominee Movie Rating by Season (1927-2010)", fontweight='bold')
plt.xlabel('Movie Rating', fontweight='bold')
plt.ylabel("Count of Movie Rating", fontweight='bold')

plt.xticks([r + barWidth for r in range(len(height_winter_rated))], ['R', 'PG', 'Not Rated', 'PG-13', 'Approved', 'Passed', 'G'])

#create legend, show graphic, and push to .png
plt.legend()
plt.savefig("count_of_rated_by_season.png")
plt.show()